# Init SparkContext

In [1]:
import os
from datetime import datetime
from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as sf

In [2]:
spark = (SparkSession.builder.appName("tpch-benchmark-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Load and create Catalog table

In [3]:
spark.sql("SHOW databases").show()


+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
spark.sql("SHOW tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [5]:
ls_files = [
    "h_customer.dsv",
    "h_lineitem.dsv",
    "h_nation.dsv",
    "h_order.dsv",
    "h_part.dsv",
    "h_partsupp.dsv",
    "h_region.csv",
    "h_supplier.dsv",
]

In [ ]:
base_path = "s3a://warehouse/tpch_data"

for file in ls_files:
    file_name, ext = file.split(".")
    prefix, table_name = file_name.split("_")
    print("Loading", table_name)
    
    delimeter = "," if ext == "csv" else "|"
    df_data = (
        spark.read.option("delimiter", delimeter)
        .option("header", True)
        .option("inferSchema" , True)
        .csv(os.path.join(base_path, file))
    )
    
    # date convert
    for col in df_data.columns:
        if "date" in col.lower():
            df_data = df_data.withColumn(col, sf.date_sub(sf.to_date(col, "dd.MM.yy"), 365 * 100))
    
    # print info
    df_data.printSchema()
    display(df_data.limit(5).toPandas())
    
    # register temporary view
    df_data.createOrReplaceTempView(table_name)

Loading customer
root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: string (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,3087,Customer#0000003087,R6k2NmyigwP0gPNR2yg2226g2RN,16,117-398-8428,"6168,09",HOUSEHOLD,blithely final theodolites try to hang ironica...
1,3089,Customer#0000003089,00NB2ywNkRg6kwkgk,9,626-408-9521,"9423,01",BUILDING,permanently close tithes are of .
2,3092,Customer#0000003092,2kmPi6Bm m6wkwk2RRB2kgi0yyLyPk,12,330-709-7828,"1653,94",AUTOMOBILE,quietly even somas beside thrash fluffily at .
3,3093,Customer#0000003093,g Nii PkLw2mwBNP2Ni04g04PRPLN0Rk6Pi4,18,656-420-2897,"9718,37",FURNITURE,blithely pending pinto beans nag blithely .
4,3100,Customer#0000003100,242y4 w264y4B0Pg 60yiwNgLkN6R0wkywkPmii,23,424-440-1544,"5945,43",FURNITURE,express requests was quickly above .


Loading lineitem


In [13]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         | customer|      false|
|         | lineitem|      false|
|         |   nation|      false|
|         |    order|      false|
|         |     part|      false|
|         | partsupp|      false|
|         |   region|      false|
|         | supplier|      false|
+---------+---------+-----------+



# Benchmark

## 1. Group by, Order by

In [18]:
%%timeit -r 4

sql_stm = """
-- 1
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus;
"""
spark.sql(sql_stm).limit(20).toPandas()

2min 58s ± 24.2 s per loop (mean ± std. dev. of 4 runs, 1 loop each)


## Compressed and repartition files

In [14]:
spark.sql("select * from lineitem").repartition(6).write.parquet("s3a://warehouse/outputs/lineitem")                    

AnalysisException: path s3a://warehouse/outputs/lineitem already exists.

In [16]:
# register temporary view
df_data = spark.read.parquet("s3a://warehouse/outputs/lineitem")
df_data.createOrReplaceTempView("lineitem_parts")

In [17]:
%%timeit -r 4

sql_stm = """
-- 1
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem_parts
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus;
"""
spark.sql(sql_stm).limit(20).toPandas()

53.9 s ± 1.18 s per loop (mean ± std. dev. of 4 runs, 1 loop each)


## 2. Join, Group By, Order By

In [ ]:
%%timeit -r 4

sql_stm = """
--3
select
	l_orderkey,
	sum(l_extendedprice * (1 - l_discount)) as revenue,
	o_orderdate,
	o_shippriority
from
	customer,
	order,
	lineitem
where
	c_mktsegment = 'AUTOMOBILE'
	and c_custkey = o_custkey
	and l_orderkey = o_orderkey
group by
	l_orderkey,
	o_orderdate,
	o_shippriority
order by
	revenue desc,
	o_orderdate
"""
spark.sql(sql_stm).show()

## 3. Sub queries, Join, Group by, Order by

In [ ]:
%%timeit -r 4

sql_stm = """
select
	supp_nation,
	cust_nation,
	l_year,
	sum(volume) as revenue
from
	(
		select
			n1.n_name as supp_nation,
			n2.n_name as cust_nation,
			extract(year from l_shipdate) as l_year,
			l_extendedprice * (1 - l_discount) as volume
		from
			supplier,
			lineitem,
			order,
			customer,
			nation n1,
			nation n2
		where
			s_suppkey = l_suppkey
			and o_orderkey = l_orderkey
			and c_custkey = o_custkey
			and s_nationkey = n1.n_nationkey
			and c_nationkey = n2.n_nationkey
			and l_shipdate between date '1995-01-01' and date '1996-12-31'
	) as shipping
group by
	supp_nation,
	cust_nation,
	l_year
order by
	supp_nation,
	cust_nation,
	l_year;
"""
spark.sql(sql_stm).show()